In [2]:
import tensorflow as tf

/Users/rolandschwaiger/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [8]:
x=tf.Variable(3,name="x")
y=tf.Variable(4,name="y")
f=x*x*y + y +2

In [9]:
print(f)

Tensor("add_2:0", shape=(), dtype=int32)


In [10]:
sess=tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result=sess.run(f)
print(result)


42


In [11]:
sess.close()

In [13]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
print(result)    

42


In [15]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()

In [17]:
sess.close()

In [18]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

In [19]:
with tf.Session() as sess:
    print(y.eval()) 
    print(z.eval())

10
15


In [20]:
with tf.Session() as sess:
    y_val,z_val = sess.run([y,z])
    print(y_val)
    print(z_val)

10
15


In [24]:
import numpy as np
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
m,n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)),housing.data]

X=tf.constant(housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
XT=tf.transpose(X)
theta=tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)
with tf.Session() as sess:
    theta_value = theta.eval()

In [35]:
from sklearn.preprocessing import StandardScaler

n_epochs = 2000
learning_rate = 0.01
batch_size = 100
scaler = StandardScaler()

scaler.fit(housing_data_plus_bias)
scaled_housing_data_plus_bias = scaler.transform(housing_data_plus_bias)

X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
#X=tf.placeholder(tf.float32,shape=(None,n+1),name="X")
#y=tf.placeholder(tf.float32,shape=(None,1),name="y")
theta=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name="theta")
y_pred=tf.matmul(X,theta,name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name="mse")
#gradients=2/m * tf.matmul(tf.transpose(X),error)
#training_op = tf.assign(theta,theta-learning_rate*gradients)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(mse)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch",epoch,"MSE =",mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

Epoch 0 MSE = 7.08612
Epoch 100 MSE = 4.95629
Epoch 200 MSE = 4.89878
Epoch 300 MSE = 4.87235
Epoch 400 MSE = 4.85365
Epoch 500 MSE = 4.84009
Epoch 600 MSE = 4.83022
Epoch 700 MSE = 4.82305
Epoch 800 MSE = 4.81781
Epoch 900 MSE = 4.81399
Epoch 1000 MSE = 4.81119
Epoch 1100 MSE = 4.80914
Epoch 1200 MSE = 4.80763
Epoch 1300 MSE = 4.80653
Epoch 1400 MSE = 4.80571
Epoch 1500 MSE = 4.80511
Epoch 1600 MSE = 4.80466
Epoch 1700 MSE = 4.80432
Epoch 1800 MSE = 4.80407
Epoch 1900 MSE = 4.80388


In [38]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris = load_iris()
X = iris.data[:,(2,3)] #petal length, width
y = (iris.target == 0).astype(np.int) #Iris Setosa?
per_clf = Perceptron(random_state = 42)
per_clf.fit(X,y)
y_pred = per_clf.predict([[2,0.5]])
print(y_pred)


[1]


/Users/rolandschwaiger/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [40]:
import tensorflow as tf

feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X)